<a href="https://colab.research.google.com/github/CS23M005/DL_A1/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import wandb

import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist


from FFNN import *

(X_train1, y_train1), (X_test, y_test) = fashion_mnist.load_data()

kk = len(X_train1)
xt = np.split(X_train1[0:kk], [int(len(X_train1[0:kk])*0.9)])
yt = np.split(y_train1[0:kk], [int(len(y_train1[0:kk])*0.9)])
X_train = xt[0]/255
X_validation = xt[1]/255
y_train = yt[0]
y_validation = yt[1]


sweep_config = {"name": "complete-sweep-25dw", "method": "random"}
sweep_config["metric"] = {"name": "validation_acc", "goal": "maximize"}
parameters_dict = {
                "num_epochs": {"values": [2, 3, 4, 5]}, \
                "num_hidden_layers": {"values": [2, 3, 4]}, \
                "size_hidden_layer": {"values": [16, 32, 64, 128]}, \
                "learning_rate": {"values": [ 1e-2, 1e-3, 1e-4]}, \
                "optimizer": {"values": ["sgd", "adam", "mgd", "nadam", "rmsprop", "nag"]}, \
                "batch_size": {"values": [32, 64, 128]}, \
                "weight_init": {"values": ["random", "xavier"]} , \
                "activation": {"values": ["sigmoid", "tanh", "relu"]}, \
                "loss": {"values": ["crossentropy"]}, \
                "reg_lambda": {"values": [0.001, 0.0005, 0]}, \
                  }
sweep_config["parameters"] = parameters_dict



sweep_id = wandb.sweep(sweep_config,project='Assignment1', entity='cs23m005')

def train_nn():
    config_defaults = {
            "num_epochs": 3, \
            "num_hidden_layers":  4, \
            "size_hidden_layer": 128, \
            "learning_rate": 1e-3, \
            "optimizer": "mgd", \
            "batch_size": 32, \
            "weight_init": "xavier" , \
            "activation": "tanh", \
            "loss": "crossentropy", \
            "reg_lambda": 0.001, \
    }
    wandb.init(config = config_defaults, project = "Assignment1")
    config = wandb.config
    print(config.optimizer)
    wandb.run.name = "e_{}_hl_{}_opt_{}_bs_{}_init_{}_ac_{}_loss_{}".format(config.num_epochs,\
                                                                  config.num_hidden_layers,\
                                                                  config.size_hidden_layer,\
                                                                  config.learning_rate,\
                                                                  config.optimizer,\
                                                                  config.batch_size,\
                                                                  config.weight_init,\
                                                                  config.activation,\
                                                                  config.loss,\
                                                                  config.reg_lambda)

    seed(42)
    n_inputs = 784
    n_layers = config.num_hidden_layers
    n_neurons = config.size_hidden_layer
    n_outputs = 10
    init_mode = config.weight_init
    w,b = init_nn(n_inputs, n_layers, n_neurons, n_outputs, init_mode)
    w_old = w.copy()
    b_old = b.copy()

    eta = config.learning_rate
    activation_f = config.activation
    batch_size = config.batch_size
    loss = config.loss
    ud_lambda = config.reg_lambda
    optimizer = config.optimizer

    beta = 0.9
    beta1 = 0.9
    beta2 = 0.999

    eps = 1e-9
    max_epochs = config.num_epochs
    momentum = 0.9

    if optimizer == "sgd":
      w_n,b_n, a_n, h_n = sgd(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, True, max_epochs)
    elif optimizer == "mgd":
      w_n,b_n, a_n, h_n = mgd(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, max_epochs, momentum)
    elif optimizer == "nag":
      w_n,b_n, a_n, h_n = nag(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, True, max_epochs, momentum)
    elif optimizer == "rmsprop":
      w_n,b_n, a_n, h_n = rmsprop(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, True, max_epochs, beta, eps)
    elif optimizer == "adam":
      w_n,b_n, a_n, h_n = adam(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, True, max_epochs, beta1, beta2, eps)
    elif optimizer == "nadam":
      w_n,b_n, a_n, h_n = nadam(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, True, max_epochs, beta1, beta2, eps)


    train_accuracy, yp = accuracy(X_train, y_train, w_n, b_n, n_layers, activation_f)
    validation_accuracy, yp = accuracy(X_validation, y_validation, w_n, b_n, n_layers, activation_f)
    test_accuracy, ytp = accuracy(X_test, y_test, w_n, b_n, n_layers, activation_f)


    wandb.log({"train_acc": train_accuracy, \
               "validation_acc": validation_accuracy, \
                "test_acc": test_accuracy, \
                })
    print("accu: ", validation_accuracy)


####################################################################

#sweep_id = wandb.sweep(sweep_config, project = "Assignment1")
#wandb.agent(sweep_id, function = train_nn, count=25)
# wandb.finish()
#sweep_id = wandb.sweep(project = "Assignment1")
#wandb.agent(function = train_nn)
#wandb.finish()
#train_nn()
#wandb.finish()

if __name__ == "__main__":
    __spec__ = "ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>)"
    wandb.agent(sweep_id, train_nn, count = 1)
    wandb.finish()
